In [20]:
from spyral.core.constants import QBRHO_2_P

from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.target import GasTarget, load_target
from spyral_utils.plot import Histogrammer

from pathlib import Path
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import vector

import matplotlib.backends.backend_pdf as be_pdf
%matplotlib widget

In [21]:
# Load config
workspace_path = Path("/Volumes/e20009/e20009_analysis")
solver_result_path = workspace_path / "InterpSolver"
target_material_path = Path("/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/e20009_target.json")
beam_events_path = workspace_path / "beam_events"

# Run number range (inclusive)
run_min = 108
run_max = 366

# The nucleus we observe (the one fitted)
ejectile_z = 1
ejectile_a = 1

# The incoming nucleus (the beam)
projectile_z = 4
projectile_a = 10

# The target nucleus
target_z = 1
target_a = 2

residual_z = target_z + projectile_z - ejectile_z
residual_a = target_a + projectile_a - ejectile_a

if residual_z < 0:
    raise Exception(f"Illegal nuclei! Residual Z: {residual_z}")
if residual_a < 1:
    raise Exception(f"Illegal nuclei! Residual A: {residual_a}")

In [22]:
# Setup nuclear data objects
nuclear_map = NuclearDataMap()

target_material = load_target(target_material_path, nuclear_map)
if not isinstance(target_material, GasTarget):
    print('Target error!')

ejectile = nuclear_map.get_data(ejectile_z, ejectile_a)
projectile = nuclear_map.get_data(projectile_z, projectile_a)
target = nuclear_map.get_data(target_z, target_a)
residual = nuclear_map.get_data(residual_z, residual_a)

# Initial beam energy
proj_energy_start = 93.5 #MeV

In [23]:
# Define parameters used for analysis
min_z: float = 0.004    # Units of meters. Minimum z value of vertex
max_z: float = 0.958    # Units of meters. Maximum z value of vertex

hist_low: float = -0.5
hist_high: float = 4.5
hist_bins: int = 90

redchi: float = 5  # Events must have a reduced chi squared value less than this

In [24]:
# Make function to create angular histogram for input CM angular range

# Construct target vector
target_vector = vector.array({"px": [0.0], "py": [0.0], "pz": [0.0], "E": [target.mass]})

def make_angular_hist(ang_low: float, ang_high: float):
    # Make histogram to store angular cut
    grammer = Histogrammer()
    grammer.add_hist1d('ex', hist_bins, (hist_low, hist_high))

    for run in range(run_min, run_max+1):
        df = None
        try:
            path = solver_result_path / f"run_{run:04d}_{ejectile.isotopic_symbol}.parquet"
            df = pl.read_parquet(path)
        except Exception:
            continue
        
        # Apply gates to data
        df = (df.filter((pl.col('redchisq') < redchi)
                    & (pl.col("vertex_z") > min_z) 
                    & (pl.col("vertex_z") < max_z)
                    )
                .sort("polar", descending=True)
                .unique("event", keep="first")
    )

        # Construct the projectile vectors (beam)
        vertices = df.select(['vertex_x', 'vertex_y', 'vertex_z']).to_numpy()
        distances = np.linalg.norm(vertices, axis=1)
        projectile_ke = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, distances)
        projectile_vector = vector.array({
            "px": np.zeros(len(projectile_ke)),
            "py": np.zeros(len(projectile_ke)),
            "pz": np.sqrt(projectile_ke * (projectile_ke + 2.0 * projectile.mass)),
            "E": projectile_ke + projectile.mass
        })

        # Construct the ejectile vectors (detected)
        momentum = df.select('brho').to_numpy().flatten() * float(ejectile.Z) * QBRHO_2_P
        polar = df.select('polar').to_numpy().flatten()
        az = df.select('azimuthal').to_numpy().flatten()
        ejectile_vector = vector.array({
            "px": momentum * np.sin(polar) * np.cos(az),
            "py": momentum * np.sin(polar) * np.sin(az),
            "pz": momentum * np.cos(polar),
            "E": np.sqrt(momentum**2.0 + ejectile.mass**2.0)
        })

        # Do the kinematics
        residual_vector = target_vector + projectile_vector - ejectile_vector # type: ignore
        ex_energy = residual_vector.mass - residual.mass # Excitation energy is "extra" mass

        # Calculate CM scattering angle 
        cm_vec = ejectile_vector.boostCM_of(projectile_vector + target_vector)
        cm_polar = np.pi - cm_vec.theta

        # Make mask for CM angle cut
        mask = (np.deg2rad(ang_low) <= cm_polar) & (cm_polar < np.deg2rad(ang_high))        # Make note of asymmetric inequalites for binning!
        ex_energy_gated = ex_energy[mask]

        grammer.fill_hist1d('ex', ex_energy_gated)

    return grammer

In [25]:
# Make array of edges of angular cuts
cuts = np.linspace(0, 40, 21)

# Make pdf to store all images
pdf = be_pdf.PdfPages('angular_cuts.pdf')

# Create angular cuts
for angle in range(len(cuts)-1):
    low_edge = cuts[angle]
    high_edge = cuts[angle + 1]
    grammer = make_angular_hist(low_edge,high_edge)
    
    # Plot angular histogram and save it
    ex_hist = grammer.get_hist1d("ex")
    fig, ax = plt.subplots()
    ax.stairs(ex_hist.counts, edges=ex_hist.bins)
    ax.set_title(fr"{low_edge}$^\circ$ - {high_edge}$^\circ$ c.m.")
    ax.set_xlabel("Excitation Energy (MeV)")
    ax.set_ylabel("Counts")
    pdf.savefig(fig)
    plt.close()
pdf.close()
